此脚本用于Grep命令（grep -aiE "'03046.*2022-09-" *.txt -B1 | grep -aiE "^T.*2022-" > status.csv）之后

第一步将报警信息附加到状态变化信息之后

第二步插值，在状态变化之前3s插入上个变化的信息，如果两个状态变化跨越自然小时，则在整点前后各插入上个状态变化的信息

In [298]:
import pandas as pd
import numpy as np
#import os
#import datetime
#import tkinter.filedialog as tk
#import tkinter

下面开始导入visit time数据

In [299]:
dfv = pd.read_excel(r"D:\Sync_ColorCloud\LogAnalysis\AllWerfenChinaTop\downTimeByError\Data\rawData\fseVisit.xlsx",
    usecols=["Engineer (Primary)","Visit Date","Arr.Time","Exit time","Serial Number","Labour Code"])
dfv.head()

,Engineer (Primary),Labour Code,Serial Number,Visit Date,Arr.Time,Exit time
0,ZAPE000087,10,18060322,2022-09-01,13:00:00,18:30:00
1,ZAPE000087,40,20030515,2022-09-05,12:30:00,17:00:00
2,ZAPE000087,40,20030515,2022-09-06,11:00:00,11:10:00
3,ZAPE000087,40,16050133J,2022-09-06,13:00:00,17:00:00
4,ZAPE000087,40,17080293,2022-09-07,12:30:00,17:30:00


In [300]:
rename_dic = {
    "Engineer (Primary)":"fse",
    "Model":"model",
    "Labour Code":"laborV",
    "Serial Number":"sn",
    "Visit Date":"date",
    "Arr.Time":"arrTime",
    "Exit time":"extTime",
    "Serial Number HemoCell":"fsn"}
dfv.rename(columns=rename_dic,inplace=True)
dfv.head()

,fse,laborV,sn,date,arrTime,extTime
0,ZAPE000087,10,18060322,2022-09-01,13:00:00,18:30:00
1,ZAPE000087,40,20030515,2022-09-05,12:30:00,17:00:00
2,ZAPE000087,40,20030515,2022-09-06,11:00:00,11:10:00
3,ZAPE000087,40,16050133J,2022-09-06,13:00:00,17:00:00
4,ZAPE000087,40,17080293,2022-09-07,12:30:00,17:30:00


In [301]:
dfv.dropna(inplace=True)

In [302]:
dfv["arrTime"] = pd.to_datetime(dfv.date.astype("str") + " " + dfv.arrTime.astype("str"))
dfv["extTime"] = pd.to_datetime(dfv.date.astype("str") + " " + dfv.extTime.astype("str"))
dfv.head()

,fse,laborV,sn,date,arrTime,extTime
0,ZAPE000087,10,18060322,2022-09-01,2022-09-01 13:00:00,2022-09-01 18:30:00
1,ZAPE000087,40,20030515,2022-09-05,2022-09-05 12:30:00,2022-09-05 17:00:00
2,ZAPE000087,40,20030515,2022-09-06,2022-09-06 11:00:00,2022-09-06 11:10:00
3,ZAPE000087,40,16050133J,2022-09-06,2022-09-06 13:00:00,2022-09-06 17:00:00
4,ZAPE000087,40,17080293,2022-09-07,2022-09-07 12:30:00,2022-09-07 17:30:00


In [303]:
dfv.drop("date",inplace=True,axis=1)
dfv.head()

,fse,laborV,sn,arrTime,extTime
0,ZAPE000087,10,18060322,2022-09-01 13:00:00,2022-09-01 18:30:00
1,ZAPE000087,40,20030515,2022-09-05 12:30:00,2022-09-05 17:00:00
2,ZAPE000087,40,20030515,2022-09-06 11:00:00,2022-09-06 11:10:00
3,ZAPE000087,40,16050133J,2022-09-06 13:00:00,2022-09-06 17:00:00
4,ZAPE000087,40,17080293,2022-09-07 12:30:00,2022-09-07 17:30:00


In [304]:
dfv = dfv.melt(("fse", "laborV", "sn"),var_name="visitType",value_name="time")
dfv.head()

,fse,laborV,sn,visitType,time
0,ZAPE000087,10,18060322,arrTime,2022-09-01 13:00:00
1,ZAPE000087,40,20030515,arrTime,2022-09-05 12:30:00
2,ZAPE000087,40,20030515,arrTime,2022-09-06 11:00:00
3,ZAPE000087,40,16050133J,arrTime,2022-09-06 13:00:00
4,ZAPE000087,40,17080293,arrTime,2022-09-07 12:30:00


In [305]:
dfv["visitType"] = dfv["visitType"].map({"arrTime":1,"extTime":-1})
dfv.head()

,fse,laborV,sn,visitType,time
0,ZAPE000087,10,18060322,1,2022-09-01 13:00:00
1,ZAPE000087,40,20030515,1,2022-09-05 12:30:00
2,ZAPE000087,40,20030515,1,2022-09-06 11:00:00
3,ZAPE000087,40,16050133J,1,2022-09-06 13:00:00
4,ZAPE000087,40,17080293,1,2022-09-07 12:30:00


In [306]:
dfv["sn"] = "T" + dfv["sn"]
dfv.head()

,fse,laborV,sn,visitType,time
0,ZAPE000087,10,T18060322,1,2022-09-01 13:00:00
1,ZAPE000087,40,T20030515,1,2022-09-05 12:30:00
2,ZAPE000087,40,T20030515,1,2022-09-06 11:00:00
3,ZAPE000087,40,T16050133J,1,2022-09-06 13:00:00
4,ZAPE000087,40,T17080293,1,2022-09-07 12:30:00


In [307]:
dfv.rename(columns={"visitType":"visitValue","time":"dt"},inplace=True)
dfv = dfv.loc[:,["sn","dt","fse","visitValue","laborV"]]
dfv.head()

,sn,dt,fse,visitValue,laborV
0,T18060322,2022-09-01 13:00:00,ZAPE000087,1,10
1,T20030515,2022-09-05 12:30:00,ZAPE000087,1,40
2,T20030515,2022-09-06 11:00:00,ZAPE000087,1,40
3,T16050133J,2022-09-06 13:00:00,ZAPE000087,1,40
4,T17080293,2022-09-07 12:30:00,ZAPE000087,1,40


In [308]:
dfv["dt"] = dfv["dt"] + pd.Timedelta(10,"s")
dfv.head()

,sn,dt,fse,visitValue,laborV
0,T18060322,2022-09-01 13:00:10,ZAPE000087,1,10
1,T20030515,2022-09-05 12:30:10,ZAPE000087,1,40
2,T20030515,2022-09-06 11:00:10,ZAPE000087,1,40
3,T16050133J,2022-09-06 13:00:10,ZAPE000087,1,40
4,T17080293,2022-09-07 12:30:10,ZAPE000087,1,40


In [309]:
visit_dic = dict(list(dfv.groupby("sn")))

下面开始导入call time数据

In [310]:
df_calltime = pd.read_excel(r"D:\Sync_ColorCloud\LogAnalysis\AllWerfenChinaTop\downTimeByError\Data\rawData\calltime.xlsx")
df_calltime.head()

,Call Nr.,Call date,Call Time,Completed date,Completed Time,Labour Code,Engineer name,Name,PM Type,Status srv call,Serial Number,Engineer (Primary),Call text,Created by,Changed by
0,2003961876,2022-09-12,14:17:22,NaT,00:00:00,10,ZHANG JIA JING,Wenzhou City People's Hosp.,NaN,94,16110154J,ZAPE000060,信河急诊 dd6.38 照理说你们仪器会自动补做稀释模式吧sample 2294232...,CZHANG3,ZJIAJING
1,2003969869,2022-09-20,16:30:38,2022-09-20,16:53:55,30,BI HUIJUN,Lingbi County People's Hosp.,MON,99,20080698,ZAPE000079,test,BHUIJUN,BHUIJUN
2,2003969851,2022-09-20,16:20:36,2022-09-20,16:50:17,30,BI HUIJUN,Lingbi County People's Hosp.,MON,99,20080698,ZAPE000079,test,BHUIJUN,BHUIJUN
3,2003950547,2022-09-01,10:37:00,2022-09-01,23:52:23,10,JI FU YOU,Shandong Provincial Hosp.,NaN,99,18060322,ZAPE000087,optical reading unit 3 dark readings too high,FJI,FJI
4,2003981900,2022-09-28,16:41:14,2022-09-28,14:12:32,10,JI FU YOU,Shandong Provincial Hosp.,NaN,99,18060320,ZAPE000087,1253， Cuvette Shuttle movement failure,CZHIHUA,FJI


In [311]:
df_calltime = df_calltime.iloc[:,[0,1,2,3,4,5,10]]
df_calltime.head()

,Call Nr.,Call date,Call Time,Completed date,Completed Time,Labour Code,Serial Number
0,2003961876,2022-09-12,14:17:22,NaT,00:00:00,10,16110154J
1,2003969869,2022-09-20,16:30:38,2022-09-20,16:53:55,30,20080698
2,2003969851,2022-09-20,16:20:36,2022-09-20,16:50:17,30,20080698
3,2003950547,2022-09-01,10:37:00,2022-09-01,23:52:23,10,18060322
4,2003981900,2022-09-28,16:41:14,2022-09-28,14:12:32,10,18060320


In [312]:
rename_dic1 =   {
                    "Call date":"startdate",
                    "Call Time":"starttime",
                    "Completed date":"enddate",
                    "Completed Time":"endtime",
                    "Serial Number":"sn",
                    "Call Nr.":"call",
                    "Labour Code":"laborC"
                }

df_calltime.rename(columns=rename_dic1,inplace=True)
df_calltime.head()

,call,startdate,starttime,enddate,endtime,laborC,sn
0,2003961876,2022-09-12,14:17:22,NaT,00:00:00,10,16110154J
1,2003969869,2022-09-20,16:30:38,2022-09-20,16:53:55,30,20080698
2,2003969851,2022-09-20,16:20:36,2022-09-20,16:50:17,30,20080698
3,2003950547,2022-09-01,10:37:00,2022-09-01,23:52:23,10,18060322
4,2003981900,2022-09-28,16:41:14,2022-09-28,14:12:32,10,18060320


In [313]:
df_calltime["sn"] = "T" + df_calltime["sn"]
df_calltime.head()

,call,startdate,starttime,enddate,endtime,laborC,sn
0,2003961876,2022-09-12,14:17:22,NaT,00:00:00,10,T16110154J
1,2003969869,2022-09-20,16:30:38,2022-09-20,16:53:55,30,T20080698
2,2003969851,2022-09-20,16:20:36,2022-09-20,16:50:17,30,T20080698
3,2003950547,2022-09-01,10:37:00,2022-09-01,23:52:23,10,T18060322
4,2003981900,2022-09-28,16:41:14,2022-09-28,14:12:32,10,T18060320


In [314]:
df_calltime.loc[df_calltime.enddate.isnull(),"endtime"] = pd.NaT
df_calltime.head()

,call,startdate,starttime,enddate,endtime,laborC,sn
0,2003961876,2022-09-12,14:17:22,NaT,NaT,10,T16110154J
1,2003969869,2022-09-20,16:30:38,2022-09-20,16:53:55,30,T20080698
2,2003969851,2022-09-20,16:20:36,2022-09-20,16:50:17,30,T20080698
3,2003950547,2022-09-01,10:37:00,2022-09-01,23:52:23,10,T18060322
4,2003981900,2022-09-28,16:41:14,2022-09-28,14:12:32,10,T18060320


In [315]:
df_calltime["start_time"] = df_calltime.startdate.astype("str") + " " + df_calltime.starttime.astype("str")
df_calltime["end_time"] = df_calltime.enddate.astype("str") + " " + df_calltime.endtime.astype("str")
df_calltime.head()

,call,startdate,starttime,enddate,endtime,laborC,sn,start_time,end_time
0,2003961876,2022-09-12,14:17:22,NaT,NaT,10,T16110154J,2022-09-12 14:17:22,NaT NaT
1,2003969869,2022-09-20,16:30:38,2022-09-20,16:53:55,30,T20080698,2022-09-20 16:30:38,2022-09-20 16:53:55
2,2003969851,2022-09-20,16:20:36,2022-09-20,16:50:17,30,T20080698,2022-09-20 16:20:36,2022-09-20 16:50:17
3,2003950547,2022-09-01,10:37:00,2022-09-01,23:52:23,10,T18060322,2022-09-01 10:37:00,2022-09-01 23:52:23
4,2003981900,2022-09-28,16:41:14,2022-09-28,14:12:32,10,T18060320,2022-09-28 16:41:14,2022-09-28 14:12:32


In [316]:
df_calltime = df_calltime.loc[:,["sn","start_time","end_time","call","laborC"]]
df_calltime.head()

,sn,start_time,end_time,call,laborC
0,T16110154J,2022-09-12 14:17:22,NaT NaT,2003961876,10
1,T20080698,2022-09-20 16:30:38,2022-09-20 16:53:55,2003969869,30
2,T20080698,2022-09-20 16:20:36,2022-09-20 16:50:17,2003969851,30
3,T18060322,2022-09-01 10:37:00,2022-09-01 23:52:23,2003950547,10
4,T18060320,2022-09-28 16:41:14,2022-09-28 14:12:32,2003981900,10


In [317]:
df_calltime = df_calltime.melt(id_vars=["sn","call","laborC"],value_name="dt",var_name="callValue")
df_calltime.head()

,sn,call,laborC,callValue,dt
0,T16110154J,2003961876,10,start_time,2022-09-12 14:17:22
1,T20080698,2003969869,30,start_time,2022-09-20 16:30:38
2,T20080698,2003969851,30,start_time,2022-09-20 16:20:36
3,T18060322,2003950547,10,start_time,2022-09-01 10:37:00
4,T18060320,2003981900,10,start_time,2022-09-28 16:41:14


In [318]:
df_calltime = df_calltime.loc[df_calltime.dt != "NaT NaT"]
df_calltime.head()

,sn,call,laborC,callValue,dt
0,T16110154J,2003961876,10,start_time,2022-09-12 14:17:22
1,T20080698,2003969869,30,start_time,2022-09-20 16:30:38
2,T20080698,2003969851,30,start_time,2022-09-20 16:20:36
3,T18060322,2003950547,10,start_time,2022-09-01 10:37:00
4,T18060320,2003981900,10,start_time,2022-09-28 16:41:14


In [319]:
df_calltime["dt"] = df_calltime["dt"].astype("datetime64[ns]")
df_calltime.dtypes

sn                   object
call                  int64
laborC                int64
callValue            object
dt           datetime64[ns]
dtype: object

In [320]:
callValue_dic = {
                    "start_time":1,
                    "end_time":-1
                }
df_calltime["callValue"] = df_calltime.callValue.map(callValue_dic)
df_calltime.head()

,sn,call,laborC,callValue,dt
0,T16110154J,2003961876,10,1,2022-09-12 14:17:22
1,T20080698,2003969869,30,1,2022-09-20 16:30:38
2,T20080698,2003969851,30,1,2022-09-20 16:20:36
3,T18060322,2003950547,10,1,2022-09-01 10:37:00
4,T18060320,2003981900,10,1,2022-09-28 16:41:14


In [321]:
df_calltime = df_calltime.loc[:,["sn","dt","callValue","call","laborC"]]
df_calltime.head()

,sn,dt,callValue,call,laborC
0,T16110154J,2022-09-12 14:17:22,1,2003961876,10
1,T20080698,2022-09-20 16:30:38,1,2003969869,30
2,T20080698,2022-09-20 16:20:36,1,2003969851,30
3,T18060322,2022-09-01 10:37:00,1,2003950547,10
4,T18060320,2022-09-28 16:41:14,1,2003981900,10


In [322]:
call_dic = dict(list(df_calltime.groupby("sn")))

下面开始导入ECG并合并call/visit

In [323]:
logpath = r"D:\Sync_ColorCloud\LogAnalysis\AllWerfenChinaTop\downTimeByError\Data\rawData\status.csv"

In [324]:
df1 = pd.read_csv(logpath, sep="\t", encoding="utf_8_sig", header=None, parse_dates=[2])
df1.head()

,0,1,2,3,4,5,6,7,8,9
0,T14060203.txt-'03180,INFORMATION,2022-10-01 07:50:53.520,Setup,NaN,LAS parameters modified.,NaN,0,0,255
1,T14060203.txt:'03046,INFORMATION,2022-10-01 07:51:23.470,Analyzer,NaN,Analyzer Status changed from Not connected to ...,NaN,0,0,255
2,T14060203.txt-'03106,INFORMATION,2022-10-01 07:56:46.577,Other,0,Successful log in by user: 0.,NaN,0,0,255
3,T14060203.txt:'03046,INFORMATION,2022-10-01 07:56:48.667,Analyzer,0,Analyzer Status changed from Emergency stop to...,NaN,0,0,255
4,T14060203.txt:'03046,INFORMATION,2022-10-01 07:57:24.130,Analyzer,0,Analyzer Status changed from Not connected to ...,NaN,0,0,255


In [325]:
df1.drop([1,3,4,6,7,8,9],inplace=True,axis=1)
df1.rename(columns={0 : "rowInfo", 2 : "dt", 5 : "statusChange"},inplace=True)
df1.head()

,rowInfo,dt,statusChange
0,T14060203.txt-'03180,2022-10-01 07:50:53.520,LAS parameters modified.
1,T14060203.txt:'03046,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...
2,T14060203.txt-'03106,2022-10-01 07:56:46.577,Successful log in by user: 0.
3,T14060203.txt:'03046,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...
4,T14060203.txt:'03046,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...


In [326]:
df1.dropna(inplace=True)

In [327]:
def sn_extract(strs):
    sn = strs.split(".")[0]
    return sn

In [328]:
df1.loc[:, "sn"] = df1.rowInfo.map(sn_extract)
df1.loc[:, "code"] = df1.rowInfo.str.slice(-6,)
df1.drop("rowInfo", inplace=True, axis=1)
df1.head()

,dt,statusChange,sn,code
0,2022-10-01 07:50:53.520,LAS parameters modified.,T14060203,'03180
1,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,T14060203,'03046
2,2022-10-01 07:56:46.577,Successful log in by user: 0.,T14060203,'03106
3,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,T14060203,'03046
4,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,T14060203,'03046


In [329]:
#把每条信息的后面附加上一条信息，目的是标记引起状态变化的报警
df2 = df1.copy()
df2.index += 1

In [330]:
df1 = pd.merge(df1, df2, left_index=True, right_index=True, suffixes=["","_pre"])
df1 = df1.loc[(df1.sn == df1.sn_pre) & (df1.code == "'03046"), :]
df1.drop(["statusChange_pre", "sn_pre", "code"], inplace=True, axis=1)
df1.reset_index(inplace=True, drop=True)
df1.head()

,dt,statusChange,sn,dt_pre,code_pre
0,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,T14060203,2022-10-01 07:50:53.520,'03180
1,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,T14060203,2022-10-01 07:56:46.577,'03106
2,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,T14060203,2022-10-01 07:56:48.667,'03046
3,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,T14060203,2022-10-01 07:57:24.130,'03046
4,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,T14060203,2022-10-01 07:57:35.107,'03046


In [331]:
df1 = df1.loc[:,["sn","dt","statusChange","dt_pre","code_pre"]]
df1.head()

,sn,dt,statusChange,dt_pre,code_pre
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180
1,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106
2,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046
3,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046
4,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046


In [332]:
#如果连续的两个状态变化小于5秒，则删除前一个状态变化
df1["dtNextStatus"] = df1["dt"].shift(-1)
df1["dtNextStatusSn"] = df1["sn"].shift(-1)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,dtNextStatus,dtNextStatusSn
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,2022-10-01 07:56:48.667,T14060203
1,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,2022-10-01 07:57:24.130,T14060203
2,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,2022-10-01 07:57:35.107,T14060203
3,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046,2022-10-01 07:59:59.253,T14060203
4,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046,2022-10-01 08:04:03.687,T14060203


In [333]:
df1 = df1.loc[df1.sn == df1.dtNextStatusSn, :]
df1 = df1.loc[df1["dtNextStatus"]-df1["dt"]>pd.Timedelta(2,"s"), :]
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,dtNextStatus,dtNextStatusSn
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,2022-10-01 07:56:48.667,T14060203
1,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,2022-10-01 07:57:24.130,T14060203
2,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,2022-10-01 07:57:35.107,T14060203
3,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046,2022-10-01 07:59:59.253,T14060203
4,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046,2022-10-01 08:04:03.687,T14060203


In [334]:
df1.drop(["dtNextStatus", "dtNextStatusSn"],inplace=True,axis=1)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180
1,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106
2,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046
3,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046
4,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046


In [335]:
ecg_dic = dict(list(df1.groupby("sn")))

In [336]:
l = []
for sn, df in ecg_dic.items():
    if sn not in (list(visit_dic.keys()) + list(call_dic.keys())):
        l.append(df)
    elif (sn in visit_dic.keys()) and (sn not in call_dic.keys()):
        dfc = pd.concat([df,visit_dic[sn]])
        dfc.sort_values(by="dt",inplace=True)
        dfc["statusChange"].fillna(method="ffill",inplace=True)
        dfc["dt_pre"].fillna(method="bfill",inplace=True)
        dfc.dropna(subset=["statusChange","dt_pre"],inplace=True,how="any")
        dfc["fse"].fillna(method="ffill",inplace=True)
        dfc["laborV"].fillna(method="ffill",inplace=True)
        dfc["visitValue"].fillna(0,inplace=True)
        dfc["visitValue"] = dfc["visitValue"].cumsum()
        flag = dfc["visitValue"].mode().iloc[0]
        if flag < 0:
            dfc["visitValue"] = dfc["visitValue"] - flag
        dfc["code_pre"].fillna("visit",inplace=True)
        l.append(dfc)
    elif (sn not in visit_dic.keys()) and (sn in call_dic.keys()):
        dfc = pd.concat([df,call_dic[sn]])
        dfc.sort_values(by="dt",inplace=True)
        dfc["statusChange"].fillna(method="ffill",inplace=True)
        dfc["dt_pre"].fillna(method="bfill",inplace=True)
        dfc.dropna(subset=["statusChange","dt_pre"],inplace=True,how="any")
        dfc["callValue"].fillna(0,inplace=True)
        dfc["callValue"] = dfc["callValue"].cumsum()
        flag = dfc["callValue"].mode().iloc[0]
        if flag < 0:
            dfc["callValue"] = dfc["callValue"] - flag
        dfc["code_pre"].fillna("call",inplace=True)
        dfc["call"].fillna(method="ffill",inplace=True)
        dfc["laborC"].fillna(method="ffill",inplace=True)
        l.append(dfc)
    else:
        dfc = pd.concat([df,visit_dic[sn],call_dic[sn]])
        dfc.sort_values(by="dt",inplace=True)
        dfc["statusChange"].fillna(method="ffill",inplace=True)
        dfc["dt_pre"].fillna(method="bfill",inplace=True)
        dfc.dropna(subset=["statusChange","dt_pre"],inplace=True,how="any")
        dfc["fse"].fillna(method="ffill",inplace=True)
        dfc["laborV"].fillna(method="ffill",inplace=True)
        dfc["visitValue"].fillna(0,inplace=True)
        dfc["visitValue"] = dfc["visitValue"].cumsum()
        flag = dfc["visitValue"].mode().iloc[0]
        if flag < 0:
            dfc["visitValue"] = dfc["visitValue"] - flag
        dfc["code_pre"].fillna("visit",inplace=True)
        dfc["callValue"].fillna(0,inplace=True)
        dfc["callValue"] = dfc["callValue"].cumsum()
        flag = dfc["callValue"].mode().iloc[0]
        if flag < 0:
            dfc["callValue"] = dfc["callValue"] - flag
        dfc["call"].fillna(method="ffill",inplace=True)
        dfc["laborC"].fillna(method="ffill",inplace=True)
        l.append(dfc)


In [337]:
df1 = pd.concat(l)

In [338]:
df1.reset_index(drop=True,inplace=True)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN
1,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN
2,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN
3,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046,NaN,NaN,NaN,NaN,NaN,NaN
4,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046,NaN,NaN,NaN,NaN,NaN,NaN


In [339]:
df2 = df1.copy()
df2.drop(["statusChange", "dt_pre", "code_pre", "fse", "visitValue", "laborV", "callValue", "call", "laborC"], inplace=True, axis=1)
df1.index += 1
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC
1,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN
3,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN
4,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046,NaN,NaN,NaN,NaN,NaN,NaN
5,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046,NaN,NaN,NaN,NaN,NaN,NaN


In [340]:
df1 = pd.merge(df1, df2, left_index=True, right_index=True,suffixes=["", "_next"])
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC,sn_next,dt_next
1,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,T14060203,2022-10-01 07:56:48.667
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,T14060203,2022-10-01 07:57:24.130
3,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN,T14060203,2022-10-01 07:57:35.107
4,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046,NaN,NaN,NaN,NaN,NaN,NaN,T14060203,2022-10-01 07:59:59.253
5,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046,NaN,NaN,NaN,NaN,NaN,NaN,T14060203,2022-10-01 08:04:03.857


In [341]:
df1 = df1.loc[df1.sn == df1.sn_next, :]
df1.drop(["sn_next"], inplace=True, axis=True)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC,dt_next
1,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:56:48.667
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:57:24.130
3,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:57:35.107
4,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:59:59.253
5,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 08:04:03.857


In [342]:
df1.reset_index(inplace=True, drop=True)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC,dt_next
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:56:48.667
1,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:57:24.130
2,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:57:35.107
3,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:59:59.253
4,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 08:04:03.857


In [343]:
replace_dic = {
    "开机": "Power up",
    "初始化": "Initializing",
    "维护": "Maintenance",
    "错误": "Error",
    "紧急停止": "Emergency stop",
    "忙": "Busy",
    "诊断": "Diagnostics",
    "准备": "Ready",
    "受控停机": "Controlled stop",
    "未连接": "Not connected",
    "温度调整": "Adjusting thermal",
    "分析仪状态从": "Analyzer Status changed from",
    "变为": "to"
}

def replace_desp(desp):
    if "分析仪状态从" not in desp:
        return desp
    for (cn, en) in replace_dic.items():
        desp = desp.replace(cn, en)
    return desp

In [344]:
df1["statusChange"] = df1["statusChange"].map(replace_desp)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC,dt_next
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:56:48.667
1,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:57:24.130
2,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:57:35.107
3,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:59:59.253
4,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 08:04:03.857


In [345]:
l = []

In [346]:
def insert_rows(df_row):
    global l
    if df_row["dt_next"] - df_row["dt"] < pd.Timedelta(3,"s"):
        l.append(df_row.to_dict())
    elif df_row.dt_next.strftime("%Y%m%d%H") == df_row["dt"].strftime("%Y%m%d%H"):
        l.append(df_row.to_dict())
        l.append({
            "sn" : df_row.sn,
            "dt" : df_row.dt_next - pd.Timedelta(2,"s"),
            "statusChange" : df_row.statusChange,
            "dt_pre" : df_row.dt_pre,
            "code_pre" : df_row.code_pre,
            "fse" : df_row.fse,
            "visitValue" : df_row.visitValue,
            "dt_next" : df_row.dt_next,
            "callValue" : df_row.callValue,
            "call" : df_row.call,
            "laborV" : df_row.laborV,
            "laborC" : df_row.laborC

        })
    else:
        l.append(df_row.to_dict())
        dt_tuple = df_row["dt"].timetuple()
        dt_next_tuple = df_row.dt_next.timetuple()
        dt_hourEnd = pd.Timestamp(dt_tuple.tm_year,dt_tuple.tm_mon,dt_tuple.tm_mday,dt_tuple.tm_hour) + pd.Timedelta('0 days 00:59:59.9')
        dt_next_hourStart = pd.Timestamp(dt_next_tuple.tm_year,dt_next_tuple.tm_mon,dt_next_tuple.tm_mday,dt_next_tuple.tm_hour)
        while dt_hourEnd < dt_next_hourStart:
            l.append({
                "sn" : df_row.sn,
                "dt" : dt_hourEnd, 
                "statusChange" : df_row.statusChange,
                "dt_pre" : df_row.dt_pre,
                "code_pre" : df_row.code_pre,
                "fse" : df_row.fse,
                "visitValue" : df_row.visitValue,
                "dt_next" : df_row.dt_next,
                "callValue" : df_row.callValue,
                "call" : df_row.call,
                "laborV" : df_row.laborV,
                "laborC" : df_row.laborC
            })
            l.append({
                "sn" : df_row.sn, 
                "dt" : dt_hourEnd + pd.Timedelta('0 days 00:00:0.11') , 
                "statusChange" : df_row.statusChange,
                "dt_pre" : df_row.dt_pre,
                "code_pre" : df_row.code_pre,
                "fse" : df_row.fse,
                "visitValue" : df_row.visitValue,
                "dt_next" : df_row.dt_next,
                "callValue" : df_row.callValue,
                "call" : df_row.call,
                "laborV" : df_row.laborV,
                "laborC" : df_row.laborC
            })
            dt_hourEnd += pd.Timedelta('0 days 01:00:00')
        l.append({
            "sn" : df_row.sn, 
            "dt" : df_row.dt_next - pd.Timedelta(2,"s"), 
            "statusChange" : df_row.statusChange,
            "dt_pre" : df_row.dt_pre,
            "code_pre" : df_row.code_pre,
            "fse" : df_row.fse,
            "visitValue" : df_row.visitValue,
            "dt_next" : df_row.dt_next,
            "callValue" : df_row.callValue,
            "call" : df_row.call,
            "laborV" : df_row.laborV,
            "laborC" : df_row.laborC
        })

In [347]:
df1.apply(insert_rows,axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
436521    None
436522    None
436523    None
436524    None
436525    None
Length: 436526, dtype: object

In [348]:
df1 = pd.DataFrame.from_records(l)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC,dt_next
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:56:48.667
1,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:56:48.667
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:57:24.130
3,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:57:24.130
4,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:57:35.107


In [349]:
df1.drop("dt_next", inplace=True, axis=True)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN
1,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN
3,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN
4,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN


In [350]:
df2 = df1.copy()
df1.index += 1
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC
1,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN
2,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN
3,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN
4,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN
5,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN


In [351]:
df2.drop(["statusChange","dt_pre","code_pre","fse","visitValue","laborV","callValue","call", "laborC"], inplace=True, axis=True)
df2.head()

,sn,dt
0,T14060203,2022-10-01 07:51:23.470
1,T14060203,2022-10-01 07:56:46.667
2,T14060203,2022-10-01 07:56:48.667
3,T14060203,2022-10-01 07:57:22.130
4,T14060203,2022-10-01 07:57:24.130


In [352]:
df1 = pd.merge(df1, df2, left_index=True, right_index=True, suffixes=["", "_next"])
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC,sn_next,dt_next
1,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,T14060203,2022-10-01 07:56:46.667
2,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,T14060203,2022-10-01 07:56:48.667
3,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,T14060203,2022-10-01 07:57:22.130
4,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,T14060203,2022-10-01 07:57:24.130
5,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN,T14060203,2022-10-01 07:57:33.107


In [353]:
df1 = df1.loc[df1.sn == df1.sn_next, :]
df1.drop("sn_next", inplace=True, axis=1)
df1.reset_index(drop=True, inplace=True)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC,dt_next
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:56:46.667
1,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:56:48.667
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:57:22.130
3,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:57:24.130
4,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-01 07:57:33.107


In [354]:
df1.loc[:,"periodH"] = (df1["dt_next"] - df1["dt"]) / pd.Timedelta(1,"h")
df1.loc[:,"date"] = df1["dt"].dt.strftime("%Y/%m/%d")
df1.loc[:,"weekday"] = df1["dt"].dt.weekday + 1
df1.loc[:,"oClock"] = df1["dt"].dt.hour

In [355]:
df1.drop("dt_next", inplace=True, axis=True)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC,periodH,date,weekday,oClock
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,0.089777,2022/10/01,6,7
1,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,0.000556,2022/10/01,6,7
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,0.009295,2022/10/01,6,7
3,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,0.000556,2022/10/01,6,7
4,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN,0.002494,2022/10/01,6,7


In [356]:
df1.loc[:, "validTimeDiff"] = (df1.dt - df1.dt_pre) / pd.Timedelta(1,"s")
df1.loc[:, "validTimeDiff"] = df1.validTimeDiff.map(lambda x: "y" if x < 2 else "n")
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC,periodH,date,weekday,oClock,validTimeDiff
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,0.089777,2022/10/01,6,7,n
1,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,0.000556,2022/10/01,6,7,n
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,0.009295,2022/10/01,6,7,n
3,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,0.000556,2022/10/01,6,7,n
4,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN,0.002494,2022/10/01,6,7,n


In [357]:
def status_extract(strs):
    if " to " in strs:
        status = strs.split(" to ")[1]
        return status[0:-1]
    else:
        return strs

In [358]:
df1.loc[:, "status"] = df1.statusChange.map(status_extract)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC,periodH,date,weekday,oClock,validTimeDiff,status
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,0.089777,2022/10/01,6,7,n,Emergency stop
1,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,NaN,NaN,NaN,NaN,NaN,0.000556,2022/10/01,6,7,n,Emergency stop
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,0.009295,2022/10/01,6,7,n,Not connected
3,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,NaN,NaN,NaN,NaN,NaN,0.000556,2022/10/01,6,7,n,Not connected
4,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,NaN,NaN,NaN,NaN,NaN,0.002494,2022/10/01,6,7,n,Power up


In [359]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310123 entries, 0 to 1310122
Data columns (total 17 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   sn             1310123 non-null  object        
 1   dt             1310123 non-null  datetime64[ns]
 2   statusChange   1310123 non-null  object        
 3   dt_pre         1310123 non-null  datetime64[ns]
 4   code_pre       1310123 non-null  object        
 5   fse            443549 non-null   object        
 6   visitValue     1164297 non-null  float64       
 7   laborV         443549 non-null   float64       
 8   callValue      831378 non-null   float64       
 9   call           319872 non-null   float64       
 10  laborC         319872 non-null   float64       
 11  periodH        1310123 non-null  float64       
 12  date           1310123 non-null  object        
 13  weekday        1310123 non-null  int64         
 14  oClock         1310123 non-null  i

In [360]:
df1.loc[df1.visitValue == 0, "fse"] = "nobody"
df1.fse.fillna("nobody",inplace=True)
df1.visitValue.fillna(0,inplace=True)
df1.callValue.fillna(0,inplace=True)
df1.laborV.fillna(0,inplace=True)
df1.laborC.fillna(0,inplace=True)
df1.call.fillna(0,inplace=True)

In [361]:
df1.loc[(df1.laborC == 10) & (df1.callValue != 0), "callValue"] = 4
df1.loc[(df1.laborC == 30) & (df1.callValue != 0), "callValue"] = 3
df1.loc[(df1.laborV == 10) & (df1.visitValue != 0), "visitValue"] = 2
df1.loc[(df1.laborV == 40) & (df1.visitValue != 0), "visitValue"] = 1

In [362]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310123 entries, 0 to 1310122
Data columns (total 17 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   sn             1310123 non-null  object        
 1   dt             1310123 non-null  datetime64[ns]
 2   statusChange   1310123 non-null  object        
 3   dt_pre         1310123 non-null  datetime64[ns]
 4   code_pre       1310123 non-null  object        
 5   fse            1310123 non-null  object        
 6   visitValue     1310123 non-null  float64       
 7   laborV         1310123 non-null  float64       
 8   callValue      1310123 non-null  float64       
 9   call           1310123 non-null  float64       
 10  laborC         1310123 non-null  float64       
 11  periodH        1310123 non-null  float64       
 12  date           1310123 non-null  object        
 13  weekday        1310123 non-null  int64         
 14  oClock         1310123 non-null  i

In [363]:
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,laborV,callValue,call,laborC,periodH,date,weekday,oClock,validTimeDiff,status
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,nobody,0.0,0.0,0.0,0.0,0.0,0.089777,2022/10/01,6,7,n,Emergency stop
1,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,nobody,0.0,0.0,0.0,0.0,0.0,0.000556,2022/10/01,6,7,n,Emergency stop
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,nobody,0.0,0.0,0.0,0.0,0.0,0.009295,2022/10/01,6,7,n,Not connected
3,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,nobody,0.0,0.0,0.0,0.0,0.0,0.000556,2022/10/01,6,7,n,Not connected
4,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,nobody,0.0,0.0,0.0,0.0,0.0,0.002494,2022/10/01,6,7,n,Power up


In [364]:
df1.to_csv(r"rawData\status_ecg.csv")